In [31]:
import subprocess
import altair as alt
import pandas as pd

In [32]:
# In order to run clingo command in the terminal and capture the output
def run_clingo(puzzle):
    result = subprocess.run(['clingo', puzzle, 'program.lp', '--stats'], 
                            capture_output=True, text=True)
    output = result.stdout
    return output

In [33]:
def analyze_output(output):
    lines = output.split('\n')
    stats = {}
    for line in lines:
        if "CPU Time" in line:
            stats["time"] = float((line.split(":")[1].strip()).split("s")[0])
        elif "Rules" in line:
            stats["rules"] = int((line.split(":")[1].strip()).split(" ")[0])
        elif "Atoms" in line:
            stats["atoms"] = int((line.split(":")[1].strip()).split(" ")[0])

    return stats

In [34]:
def classify_puzzle(stats):
    if stats["time"] < 1 and stats["atoms"] < 3000:
        return "Very easy"
    elif stats["time"] < 2:
        return "Easy"
    elif stats["atoms"] < 6000:
        return "Medium"
    elif stats["atoms"] < 10000:
        return "Hard"
    else:
        return "Harder"

In [35]:
def analyze_puzzles(puzzles):
    results = []
    for puzzle in puzzles:
        output = run_clingo(puzzle)
        stats = analyze_output(output)
        difficulty = classify_puzzle(stats)
        stats["difficulty"] = difficulty
        stats["puzzle"] = puzzle
        results.append(stats)

    return results

In [36]:
puzzles = ["program.db"]
puzzles_results = analyze_puzzles(puzzles)
df = pd.DataFrame(puzzles_results)
print(df)

   time  rules  atoms difficulty      puzzle
0   0.1  96850   2909  Very easy  program.db
